## Exercise 9 Inference of Skin Leasion

We will use a pre trained model for Skin Leasion classification.


In [ ]:
# Imports
# import libraries for simple image plotting and 
import matplotlib.pyplot as plt
import numpy as np
import torchvision.transforms as transforms
import torchvision as torchvision
from google.colab import auth
auth.authenticate_user()

In [ ]:
import torch
from torch import nn
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


In [ ]:
norm_mean = (0.485, 0.456, 0.406)
norm_std = (0.229, 0.224, 0.225)

# Standard transform we learned in the last lecture
cat_transform = transforms.Compose([
                                  # resize image to the network input size
                                  transforms.Resize((224,224)),
                                  transforms.ToTensor(),
                                  transforms.Normalize(norm_mean, norm_std),
                                   ])

In [ ]:
classes = [ 'actinic keratoses', 'basal cell carcinoma', 'benign keratosis-like lesions', 
           'dermatofibroma','melanoma', 'melanocytic nevi', 'vascular lesions']

In [ ]:
## Loading a Neural Network
model = torchvision.models.resnet18(pretrained = False) # This is a very well known network but it is designed for 1000 classes and not just cats and dogs this is why we need the next line
model.fc = nn.Linear(512,len(classes)) 

state_dict_trained = torch.hub.load_state_dict_from_url('https://github.com/IFL-CAMP/AI4MDs_21/raw/main/checkpoints/ham10k_83.pth', model_dir=".", map_location = device) # This is a checkpoint to a trained cat and dog model that works pretty well


In [ ]:
model.load_state_dict(state_dict_trained) ## Here we load the trained weights (state_dict) in our model 
model.eval() # This puts our model in eval mode

In [ ]:
!wget https://4ag46i294nta1038p13v77x1-wpengine.netdna-ssl.com/wp-content/uploads/basal-cell-carcinoma-2-open-sore.png

In [ ]:
norm_mean = (0.485, 0.456, 0.406)
norm_std = (0.229, 0.224, 0.225)

# Standard transform we learned in the last lecture
transform_ham = transforms.Compose([
                                  # resize image to the network input size
                                  transforms.Resize((224,224)),
                                  transforms.ToTensor(),
                                  transforms.Normalize(norm_mean, norm_std),
                                   ])

In [ ]:
from PIL import Image

def denorm(img):
    img[0,:,:] = (img[0,:,:] * np.asarray(norm_std[0])) + np.asarray(norm_mean[0])
    img[1,:,:] = (img[1,:,:] * np.asarray(norm_std[1])) + np.asarray(norm_mean[1])
    img[2,:,:] = (img[2,:,:] * np.asarray(norm_std[2])) + np.asarray(norm_mean[2])
    return img

def imshow(img):
    npimg = img.clone().numpy()
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.axis('off')
    ax.imshow(np.transpose(denorm(npimg), (1, 2, 0)))



In [ ]:
def show_result(outputs, leasion_tensor):
    pred = torch.argmax(outputs, dim=1).cpu().numpy() 
    imshow(torchvision.utils.make_grid(leasion_tensor))
    out_txt= f"\n--------------------------\n\n Predicted: {classes[pred[0]]}\n Class: {pred[0]}\n"
    if pred == 1 or pred == 4:
      out_txt += f" Recommendation: CHECK WITH YOUR MD!"
    else:
      out_txt += f" Recommendation: Nothing to be worried about."
    out_txt+="\n--------------------------"
    print(out_txt)

In [ ]:
fileName = "basal-cell-carcinoma-2-open-sore.png"
leasionExample = Image.open(fileName).convert('RGB')

In [ ]:
leasion_tensor = transform_ham(leasionExample)
leasion_tensor = leasion_tensor.unsqueeze(dim=0)

In [ ]:
# Lets try the model
with torch.no_grad(): # during testing we dont learn so we dont need to calculate the gradient for learning
  outputs=model(leasion_tensor) # That is a forward pass to a network
outputs = torch.exp(outputs) # transform the output to probabilities
pred = torch.argmax(outputs, dim=1).cpu().numpy()  # here we take the highest probabily and get the index 0 was cat and 1 was dog

show_result(outputs, leasion_tensor)

# Webcam example

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image as Image_py
try:
  filename_webcam = take_photo()
  print('Saved to {}'.format(filename_webcam))
  
  # Show the image which was just taken.
  display(Image_py(filename_webcam))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:

leasionExample = Image.open(filename_webcam).convert('RGB')

leasion_tensor = transform_ham(leasionExample)
leasion_tensor = leasion_tensor.unsqueeze(dim=0)

# Lets try the model
with torch.no_grad(): # during testing we dont learn so we dont need to calculate the gradient for learning
  outputs=model(leasion_tensor) # That is a forward pass to a network
pred = torch.argmax(outputs, dim=1).cpu().numpy()  # here we take the highest probabily and get the index 0 was cat and 1 was dog

show_result(outputs, leasion_tensor)
